In [1]:
import os
import pandas as pd

In [3]:
import rpy2

In [2]:
#import rpy2.robjects as robjects
#from rpy2.robjects import pandas2ri
#readRDS = robjects.r['readRDS']
#df = readRDS('df-10-znak.RDS')
#df = pandas2ri.rpy2py_dataframe(df)

In [ ]:
# Download latest month here
# http://stats.customs.gov.cn/indexEn
# http://43.248.49.97/indexEn

In [4]:
# система такая: сначала скачивается 1 файл 'downloadData.csv', он появляется в загрузках. Далее мы запускаем функцию,
# в которую вводим 3 переменные (год, месяц и экспорт/импорт - такие же, какие указали на сайте при скачивании). Функция
# преобразует csv файл к нужному формату и сохраняет либо в папке экспорт, либо в папке импорт (здесь уже указаны отзеркаленные
# данные, от лица россии), сам файл 'downloadData.csv' удаляется, таким образом, мы можем скачать новый файл с сайта и проделать ту же работу.

In [4]:
def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')
get_download_path()

'C:\\Users\\marce\\Downloads'

In [5]:
def get_table(YEAR,MONTH,TYPE): 
    # для 2017-2021 годов появляется столбец 'Unnamed: 9', для 2022 - 'Unnamed: 10',  
    # если будет ошибка, заменить 'Unnamed: 9' на 'Unnamed: 10' в строке df.drop
    d_path = get_download_path()
    filename_im = os.path.join(dirname, 'CHINA/IMPORT')
    filename_ex = os.path.join(dirname, 'CHINA/EXPORT')

    
    df = pd.read_csv(d_path + '/downloadData.csv', encoding='ISO-8859-1', on_bad_lines='skip')
    
    df = df.rename(columns={'Commodity code' : 'TNVED', 'US dollar' : 'STOIM', 'Quantity' : 'NETTO', 'Supplimentary Quantity' : 'KOL', 'Supplimentary Unit' : 'EDIZM'})
    df = df.drop({'Commodity', 'Trading partner code', 'Trading partner', 'Unit'}, axis=1)
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
    df['PERIOD'] = YEAR + '-' + MONTH

    df['TNVED'] = df['TNVED'].apply(lambda x: f'{str(x).zfill(8)}')
    df['TNVED2'] = df['TNVED'].astype(str).apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED'].astype(str).apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED'].astype(str).apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')

    #df['STOIM'] = df['STOIM'].apply(lambda x: x[:-1]).apply(lambda x: x.replace("," , "")).astype(float)
    df['STOIM'] = df['STOIM'].apply(lambda x: x.replace("," , "")).astype(float)
    df['STRANA'] = 'CH'
    if TYPE == 'ИМ':
        df['NAPR'] = 'ЭК'
        df.to_csv(filename_ex + '/' + f'data{YEAR}{str(MONTH).zfill(2)}.csv',index=False, encoding='utf-8-sig')
    elif TYPE == 'ЭК':
        df['NAPR'] = 'ИМ'
        df.to_csv(filename_im + '/' + f'data{YEAR}{str(MONTH).zfill(2)}.csv',index=False, encoding='utf-8-sig')

    os.remove(d_path + '/downloadData.csv')

In [6]:
if not os.path.exists('CHINA/EXPORT'):
    os.mkdir('CHINA/EXPORT')
if not os.path.exists('CHINA/IMPORT'):
    os.mkdir('CHINA/IMPORT')
dirname = os.getcwd()

In [65]:
YEAR = '2025'
MONTH = '05'

In [64]:
get_table(YEAR,MONTH,'ИМ')

In [62]:
get_table(YEAR,MONTH,'ЭК')